In [86]:
# stack (into time series) and seperate (different bands) the tif images of NDVI EVI from MODIS
# 
from osgeo import gdal
import numpy as np

ds = gdal.Open('/home/599/rg6346/Downloads/MODIS_NDVI_EVI/MOD13A1_005_NDVI_2000-02-18.tif')
for bi in range(ds.RasterCount):
    band = ds.GetRasterBand(bi + 1)
    # Read this band into a 2D NumPy array
    ar = band.ReadAsArray()
    print('Band %d has type %s'%(bi + 1, ar.dtype))
    raw = ar.tostring()


Band 1 has type int16
Band 2 has type int16
Band 3 has type int16


In [67]:
# loop through each band




TypeError: 'NoneType' object is not callable

In [82]:
outvrt = '/home/599/rg6346/Downloads/stacked.vrt' #/vsimem is special in-memory virtual "directory"
outtif = '/home/599/rg6346/Downloads/stacked.tif'

import glob
# dir = '/home/599/rg6346/Downloads'
tifs = glob.glob('/home/599/rg6346/Downloads/MODIS_NDVI_EVI/*.tif')

outds = gdal.BuildVRT(outvrt, tifs, separate=True)
outds = gdal.Translate(outtif, outds)
outds

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7f0154412e40> >

In [90]:
outds.RasterCount
#outds.GetRasterBand

7

In [88]:
for bi in range(outds.RasterCount):
    bandout = outds.GetRasterBand(bi + 1)
    # Read this band into a 2D NumPy array
    outar = bandout.ReadAsArray()
    print('Band %d has type %s'%(bi + 1, ar.dtype))
    raw = outar.tostring()

Band 1 has type int16
Band 2 has type int16
Band 3 has type int16
Band 4 has type int16
Band 5 has type int16
Band 6 has type int16
Band 7 has type int16


In [85]:
outar

array([[129, 129, 129, ...,   0,   0,   0],
       [129, 129, 129, ...,   0,   0,   0],
       [129, 129, 129, ...,   0,   0,   0],
       ..., 
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=int16)

In [70]:
# Update meta to reflect the number of layers
meta.update(count = len(file_list))

# Read each layer and write it to stack
with rasterio.open('stack.tif', 'w', **meta) as dst:
    for id, layer in enumerate(file_list):
        with rasterio.open(layer) as src1:
            dst.write_band(id + 1, src1.read(1))

[]

In [46]:
dst
#tifs
#file_list

<closed DatasetWriter name='stack.tif' mode='w'>

In [91]:
import xarray as xr

In [93]:
all_tifs = xr.concat(xr.open_rasterio(tif) for tif in tifs)
all_tifs

/g/data/v10/public/modules/agdc-py3-env/20171214/envs/agdc/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: the `dim` argument to `concat` will be required in a future version of xarray; for now, setting it to the old default of 'concat_dim'
  """Entry point for launching an IPython kernel.


<xarray.DataArray (concat_dims: 7, band: 3, y: 3147, x: 3346)>
array([[[[  63, ...,    0],
         ..., 
         [   0, ...,    0]],

        ..., 
        [[1014, ...,    0],
         ..., 
         [   0, ...,    0]]],


       ..., 
       [[[ 129, ...,    0],
         ..., 
         [   0, ...,    0]],

        ..., 
        [[1097, ...,    0],
         ..., 
         [   0, ...,    0]]]], dtype=int16)
Coordinates:
  * band     (band) int64 1 2 3
  * y        (y) float64 -24.59 -24.59 -24.6 -24.6 -24.6 -24.61 -24.61 ...
  * x        (x) float64 138.6 138.6 138.6 138.6 138.6 138.6 138.6 138.6 ...
Dimensions without coordinates: concat_dims
Attributes:
    crs:        +init=epsg:4326
    res:        (0.004162008945837755, 0.004162008945837755)
    is_tiled:   1
    transform:  (0.004162008945837755, 0.0, 138.5657638337764, 0.0, -0.004162...

In [94]:
tifs

['/home/599/rg6346/Downloads/MODIS_NDVI_EVI/MOD13A1_005_NDVI_2000-02-18.tif',
 '/home/599/rg6346/Downloads/MODIS_NDVI_EVI/MOD13A1_005_NDVI_2000-03-05.tif',
 '/home/599/rg6346/Downloads/MODIS_NDVI_EVI/MOD13A1_005_NDVI_2000-03-21.tif',
 '/home/599/rg6346/Downloads/MODIS_NDVI_EVI/MOD13A1_005_NDVI_2000-04-06.tif',
 '/home/599/rg6346/Downloads/MODIS_NDVI_EVI/MOD13A1_005_NDVI_2000-04-22.tif',
 '/home/599/rg6346/Downloads/MODIS_NDVI_EVI/MOD13A1_005_NDVI_2000-05-24.tif',
 '/home/599/rg6346/Downloads/MODIS_NDVI_EVI/MOD13A1_005_NDVI_2000-05-08.tif']

In [96]:
import pandas as pd

In [102]:
dates = [f.rstrip('.tif')[-10:] for f in tifs]
dates = pd.to_datetime(dates, format='%Y-%m-%d')
dates.name = 'time'
dates

DatetimeIndex(['2000-02-18', '2000-03-05', '2000-03-21', '2000-04-06',
               '2000-04-22', '2000-05-24', '2000-05-08'],
              dtype='datetime64[ns]', name='time', freq=None)

In [103]:
all_tifs = xr.concat([xr.open_rasterio(tif) for tif in tifs], dim=dates)
all_tifs

<xarray.DataArray (time: 7, band: 3, y: 3147, x: 3346)>
array([[[[  63, ...,    0],
         ..., 
         [   0, ...,    0]],

        ..., 
        [[1014, ...,    0],
         ..., 
         [   0, ...,    0]]],


       ..., 
       [[[ 129, ...,    0],
         ..., 
         [   0, ...,    0]],

        ..., 
        [[1097, ...,    0],
         ..., 
         [   0, ...,    0]]]], dtype=int16)
Coordinates:
  * band     (band) int64 1 2 3
  * y        (y) float64 -24.59 -24.59 -24.6 -24.6 -24.6 -24.61 -24.61 ...
  * x        (x) float64 138.6 138.6 138.6 138.6 138.6 138.6 138.6 138.6 ...
  * time     (time) datetime64[ns] 2000-02-18 2000-03-05 2000-03-21 ...
Attributes:
    crs:        +init=epsg:4326
    res:        (0.004162008945837755, 0.004162008945837755)
    is_tiled:   1
    transform:  (0.004162008945837755, 0.0, 138.5657638337764, 0.0, -0.004162...

In [107]:
import phenology

SyntaxError: invalid syntax (phenology.py, line 94)

In [110]:
from PIL import Image, ImageOps
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta